## Simulation of simple linear regression estimates

To illustrate some of the capabilities of `DrWatson`for simulations we consider a simple example of simulating the parameter estimates from a simple linear regression model.

In the second part of the workshop we will consider data from an experiment on the effects of sleep deprivation on reaction time.
For each subject in the study we will assume a simple linear regression model, 
$$y_i = a*x_i + b + \epsilon_i,\; i = 1,\dots,10$$
for their $i$th reaction time, $y_i$, as a function of the number of days of sleep deprivation, $x_i$.

For subject `S334` the estimates of $a$ and $b$ are approximately $a=12.25$ ms/day and $b=240.16$ ms.
An estimate of the residual standard deviation is $s=8.55$ ms, which we will use as the value of $\sigma$ in our simulation.

For all the subjects, the days of sleep deprivation are from 0 to 9.

## Parameter tuples

These parameter values are incorporated into a _named tuple_

In [1]:
pars = (a = 12.25, b = 240.16, σ = 8.55, x = 0:9)

(a = 12.25, b = 240.16, σ = 8.55, x = 0:9)

In [2]:
typeof(pars)

NamedTuple{(:a, :b, :σ, :x),Tuple{Float64,Float64,Float64,UnitRange{Int64}}}

A Julia `NamedTuple` is similar to a named list in R except that the names are _symbols_ (the symbol 'a' is written ':a') and the types of the values are part of the tuple type itself.
In `DrWatson` a simulation function typically takes a parameter tuple which is immediately _unpacked_ by calling the `@unpack` macro.
A parameter tuple can also be used to generate a meaningful file name under which to save the results of a simulation.

In [3]:
using DrWatson, LinearAlgebra, Random, PrettyTables

In [4]:
savename(pars, "arrow")  # file name for an Arrow file of results from these parameters

"a=12.25_b=240.16_σ=8.55.arrow"

## Generating a response and estimating parameters

A simple function to generate a response is

In [5]:
function oneresp(pars)
    @unpack a, b, σ, x = pars  # expand the name/value pairs
    a .* x .+ b .+ σ * randn(length(x))
end

oneresp (generic function with 1 method)

In [6]:
oneresp(pars)

10-element Array{Float64,1}:
 235.07952500038138
 241.51647464169255
 258.4024107819031
 267.7856435280689
 265.95680562786606
 300.8163593197814
 321.7553256948827
 336.3317163157975
 339.2794896297591
 344.53906645099346

A call to `randn(n)` generates an i.i.d. sample of size `n` from a standard normal distribution.
We scale these values by σ and add this "noise" to the assumed "true" response, `a * x + b`.

In Julia the "dots" ('.') before an operator like '+' or '\*' cause it to be broadcast over arrays.

### Reproducible "random" responses

It is often convenient (like when you are debugging) to be able to reproduce the "random" numbers that were generated.
To allow for this we will define another `oneresp` method that allows for a random number generator to be passed to it. 

In [7]:
function oneresp(rng::AbstractRNG, pars::NamedTuple)
    @unpack a, b, σ, x = pars
    a .* x .+ b .+ σ .* randn(rng, length(x))
end

oneresp (generic function with 2 methods)

In [8]:
rng = MersenneTwister(42);   # initialize a random number generator

In [9]:
oneresp(rng, pars)

10-element Array{Float64,1}:
 235.4059702089484
 248.6105222967121
 264.89217813997857
 274.3494110274313
 304.3607123883901
 291.62109190371854
 309.65341970746636
 327.2450266055783
 315.57097688094336
 358.98829965747854

In [10]:
y = oneresp(MersenneTwister(42), pars)  # reproduce those results

10-element Array{Float64,1}:
 235.4059702089484
 248.6105222967121
 264.89217813997857
 274.3494110274313
 304.3607123883901
 291.62109190371854
 309.65341970746636
 327.2450266055783
 315.57097688094336
 358.98829965747854

### Estimates from a single sample

The easiest way to obtain the least squares estimates is by building the model matrix and using the "backslash" operator, '\'.

In [11]:
X = hcat(ones(10), 0:9)

10×2 Array{Float64,2}:
 1.0  0.0
 1.0  1.0
 1.0  2.0
 1.0  3.0
 1.0  4.0
 1.0  5.0
 1.0  6.0
 1.0  7.0
 1.0  8.0
 1.0  9.0

In [12]:
coef = X \ y

2-element Array{Float64,1}:
 238.90883010866935
  12.03576239399893

And $s^2$, the estimated residual variance, is

In [13]:
s² = sum(abs2, y .- X * coef) / (size(X, 1) - size(X, 2))

114.91981195054993

(`abs2(x)` returns `x*x` when `x` is a real (as in, non-complex) number.)

### Simulating N parameter estimates

At this point we can extend `oneresp` to do the calculations and return a NamedTuple

In [14]:
function onepars(rng::AbstractRNG, pars::NamedTuple)
    @unpack a, b, σ, x = pars
    X = hcat(ones(length(x)), x)
    n, p = size(X)
    y = a .* x .+ b .+ σ .* randn(rng, n)
    coef = X \ y
    (slope = last(coef), intercept = first(coef), s² = sum(abs2, y .- X * coef)/(n - p))
end

onepars (generic function with 1 method)

In [15]:
onepars(MersenneTwister(42), pars)

(slope = 12.03576239399893, intercept = 238.90883010866935, s² = 114.91981195054993)

To generate a sample of these parameter estimates we first initialize a random number generator

In [16]:
rng = MersenneTwister(6354789);
samp = [onepars(rng, pars) for i in 1:100]

100-element Array{NamedTuple{(:slope, :intercept, :s²),Tuple{Float64,Float64,Float64}},1}:
 (slope = 11.239335089889355, intercept = 242.8177243543006, s² = 27.864399149440857)
 (slope = 11.970257146559899, intercept = 241.32913529965157, s² = 73.3114146323817)
 (slope = 13.639242254929487, intercept = 229.61345586372468, s² = 106.70650295502332)
 (slope = 12.900776376672322, intercept = 237.82729770208044, s² = 41.95990076259141)
 (slope = 15.24199627168337, intercept = 224.92389520814814, s² = 112.80185067488046)
 (slope = 12.811584357688236, intercept = 240.75844952653952, s² = 64.55037032600164)
 (slope = 11.970077253742533, intercept = 242.90599134310474, s² = 96.49547771466077)
 (slope = 11.34362793800242, intercept = 244.7740356685669, s² = 55.44348721003346)
 (slope = 11.877827287849142, intercept = 241.62964082235123, s² = 76.55214559549064)
 (slope = 12.807874932977553, intercept = 236.18500111592968, s² = 19.08588966027339)
 (slope = 11.987420524345353, intercept = 241.75337

It may seem like a waste to repeat the names for each result but, in fact, the names are only stored once for this vector.

In [17]:
typeof(samp)

Array{NamedTuple{(:slope, :intercept, :s²),Tuple{Float64,Float64,Float64}},1}

A vector of `NamedTuple`s is a row-oriented `Table`, as defined inthe `Tables` package.
A column-oriented table, such as a `DataFrame`, could be a `NamedTuple` of vectors.
Both types can be shown with `pretty_table` from the `PrettyTables` package.

In [18]:
pretty_table(samp)

┌─────────┬───────────┬─────────┐
│   slope │ intercept │      s² │
│ Float64 │   Float64 │ Float64 │
├─────────┼───────────┼─────────┤
│ 11.2393 │   242.818 │ 27.8644 │
│ 11.9703 │   241.329 │ 73.3114 │
│ 13.6392 │   229.613 │ 106.707 │
│ 12.9008 │   237.827 │ 41.9599 │
│  15.242 │   224.924 │ 112.802 │
│ 12.8116 │   240.758 │ 64.5504 │
│ 11.9701 │   242.906 │ 96.4955 │
│ 11.3436 │   244.774 │ 55.4435 │
│ 11.8778 │    241.63 │ 76.5521 │
│ 12.8079 │   236.185 │ 19.0859 │
│ 11.9874 │   241.753 │ 257.655 │
│ 12.3546 │    239.12 │ 76.7329 │
│ 10.5583 │   246.696 │ 48.7025 │
│ 12.7938 │    230.55 │ 104.929 │
│ 12.8718 │   237.081 │ 70.3809 │
│ 12.1422 │   245.316 │  50.597 │
│ 11.1573 │   244.447 │ 73.3371 │
│ 12.4879 │   241.083 │ 63.2412 │
│ 12.6231 │   235.876 │ 125.384 │
│ 12.7489 │    239.39 │ 29.3548 │
│  12.974 │   236.665 │ 72.9049 │
│    ⋮    │     ⋮     │    ⋮    │
└─────────┴───────────┴─────────┘
                  79 rows omitted


## Tuning the simulation

So, this approach works and produces simulated values that behave as expected.

However, we are repeating a lot of effort for each evaluation of the simulation loop.
At each iteration we create the same model matrix (of the same size) and the fixed part of the response, $\mathbf{y}$.
And in the background the expression `X \ y` will copy `X` and `y` and decompose the copy of `X` into a "QR" decomposition.

As is true in many languages, we can avoid many of these steps with enough effort in Julia.
However, unlike other languages like R or Python, we do not need to "drop down" to a compiled language like C or C++ to take advantage of the low-level features in Julia.
The magic of Julia is that the language offers you such a wide range of capabilities from high level to very low level using the same tools throughout.

An optimized version of a simulation function could look like

In [19]:
function simslr(rng::AbstractRNG, pars, N)
    @unpack a, b, σ, x = pars
    n = length(x)
    qrfac = qr!(hcat(ones(length(x)), x))
    Qt = qrfac.Q'
    R = UpperTriangular(qrfac.R)
    y₀ = convert(Vector{eltype(qrfac)}, a .* x .+ b)
    y = similar(y₀)
    cv = view(y, 1:2)   # view of the part of Q'y defining the coefficients
    rv = view(y, 3:n)   # view of the part of Q'y defining the residuals
    map(1:N) do i
        for j in axes(y, 1)
            y[j] = y₀[j] + σ * randn(rng)
        end
        lmul!(Qt, y)
        ldiv!(R, cv)
        (slope = y[2], intercept = y[1], s² = sum(abs2, rv)/(n - 2))
    end
end

simslr (generic function with 1 method)

To ensure that it works

In [20]:
rng = MersenneTwister(6354789);
samp2 = simslr(rng, pars, 100);
pretty_table(samp2)

┌─────────┬───────────┬─────────┐
│   slope │ intercept │      s² │
│ Float64 │   Float64 │ Float64 │
├─────────┼───────────┼─────────┤
│ 11.2393 │   242.818 │ 27.8644 │
│ 11.9703 │   241.329 │ 73.3114 │
│ 13.6392 │   229.613 │ 106.707 │
│ 12.9008 │   237.827 │ 41.9599 │
│  15.242 │   224.924 │ 112.802 │
│ 12.8116 │   240.758 │ 64.5504 │
│ 11.9701 │   242.906 │ 96.4955 │
│ 11.3436 │   244.774 │ 55.4435 │
│ 11.8778 │    241.63 │ 76.5521 │
│ 12.8079 │   236.185 │ 19.0859 │
│ 11.9874 │   241.753 │ 257.655 │
│ 12.3546 │    239.12 │ 76.7329 │
│ 10.5583 │   246.696 │ 48.7025 │
│ 12.7938 │    230.55 │ 104.929 │
│ 12.8718 │   237.081 │ 70.3809 │
│ 12.1422 │   245.316 │  50.597 │
│ 11.1573 │   244.447 │ 73.3371 │
│ 12.4879 │   241.083 │ 63.2412 │
│ 12.6231 │   235.876 │ 125.384 │
│ 12.7489 │    239.39 │ 29.3548 │
│  12.974 │   236.665 │ 72.9049 │
│    ⋮    │     ⋮     │    ⋮    │
└─────────┴───────────┴─────────┘
                  79 rows omitted


A superficial comparison shows that the two samples are similar.
A more detailed comparison, say by converting each of them to `DataFrame`s and using `isapprox`, would show they are the same samples.

However, their execution times and the amount of storage used when generating, say, 10,000 samples are very different.

In [21]:
rng = MersenneTwister(6354789)
@time [onepars(rng, pars) for i in 1:10_000];

  0.095813 seconds (488.83 k allocations: 41.115 MiB, 10.72% gc time)


In [22]:
rng = MersenneTwister(6354789)
@time simslr(rng, pars, 10_000);

  0.005245 seconds (10.02 k allocations: 1.146 MiB)


The optimized version is over 10 times faster and requires much less memory and fewer allocations than the original version.

The savings in memory allocations and usage come from the fact that all of the memory to be used, except for the storage of the result itself, is allocated before entering the simulation loop and re-used within the loop.
Unlike R, Julia allows a function to modify its arguments if they are composite structures like vectors.
It is customary to append "!" to the names of such _mutating_ functions, like `lmul!` (multiply, in-place, on the left) and `ldiv!` (divide, in-place, on the left).

Also, in the simulation loop itself, the vector `y` is filled in with the fixed part of the response plus the random noise in an inner loop.
This is the exact opposite of the programming style favored in R and Python where loops are avoided at all cost.
In Julia a loop is a perfectly fine way of programming an iterative operation; there is no need to perform contorsions to "vectorize" operations.